# CSC337: Coursework 1

## Task 1: Design 2
### Box plots to show the age of global power plants by primary fuel type
<b>Visual Design Type:</b> Box plot

<b>Name of Tool:</b> Altair

<b>Country:</b> Global aggregate

<b>Year:</b> N/A

<b>Visual Mappings:</b>
+ Box plots show the distribution of power plant age for each fuel type;
    + the whiskers show <i>relative</i> minimum/maximum ages of the plants

    + the filled box illustrates the IQR of the age
    
    + the black line in the box indicates the median age
    
+ Colour is used to discern renewable from non-renewable energy sources

Additionally, the following points were deliberated:
+ Whiskers do not show true min/max values for the plants, as extremely old plants were skewing the distributions. Instead, the whiskers show plants within an age of 0.5x the IQR for the fuel type

In [37]:
import pandas as pd
import geopandas as gpd
import altair as alt
import datetime as dt
import json

Debug options

In [38]:
# font to use for chart labels
__CHART_FONT__ = 'Circular'

# DEBUG: disable maximum row prevention (cripples chart performance)
alt.data_transformers.disable_max_rows()

# DEBUG: set max rows/columns in pandas table previewer
# pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

Load the GPPD data set and sequentially derive the required information

In [39]:
# load GPPD data set into pandas array
d_plants = pd.read_csv('../data/global_power_plant_database.csv')

# keep only necessary columns
d_boxplot = d_plants[['commissioning_year', 'primary_fuel']]

# retrieve GeoJSON data
with open('world.json') as world:
    world_map_data = json.load(world)

# keep only region data
d_regions = gpd.GeoDataFrame.from_features(
    world_map_data
)#[['iso_a3', 'subregion']]

d_boxplot['region'] = d_plants.merge(
    d_regions,
    how='left',
    left_on='country',
    right_on='iso_a3'
)[['region_wb']]


# remove null data from combined column
d_boxplot.dropna(
    subset=['commissioning_year', 'region'],
    inplace=True
)

# remove plants of type storage because they don't generate power
d_boxplot = d_boxplot[d_boxplot['primary_fuel'] != 'Storage']

# calculate age of plants
d_boxplot['age'] = dt.datetime.now().year - d_boxplot['commissioning_year']

d_regions

# # colour the box plots based on whether fuel type is renewable
# d_boxplot['Renewable'] = d_boxplot['primary_fuel'].isin(['Biomass', 'Geothermal', 'Hydro', 'Solar', 'Wind'])

# # replace renewability True/False with Yes/No for better presentation
# d_boxplot.replace(
#     {True: 'Yes', False: 'No'},
#     inplace=True
# )

,geometry,scalerank,featurecla,labelrank,sovereignt,sov_a3,adm0_dif,level,type,admin,...,continent,region_un,subregion,region_wb,name_len,long_len,abbrev_len,tiny,homepart,filename
0,"POLYGON ((-69.89912 12.45200, -69.89570 12.423...",3,Admin-0 country,5,Netherlands,NL1,1,2,Country,Aruba,...,North America,Americas,Caribbean,Latin America & Caribbean,5,5,5,4,-99,ABW.geojson
1,"POLYGON ((-63.00122 18.22178, -63.16001 18.171...",1,Admin-0 country,6,United Kingdom,GB1,1,2,Dependency,Anguilla,...,North America,Americas,Caribbean,Latin America & Caribbean,8,8,4,-99,-99,AIA.geojson
2,"MULTIPOLYGON (((-61.71606 17.03701, -61.74814 ...",1,Admin-0 country,6,Antigua and Barbuda,ATG,0,2,Sovereign country,Antigua and Barbuda,...,North America,Americas,Caribbean,Latin America & Caribbean,17,19,6,4,1,ATG.geojson
3,"MULTIPOLYGON (((-73.02686 21.19238, -73.05874 ...",1,Admin-0 country,4,The Bahamas,BHS,0,2,Sovereign country,The Bahamas,...,North America,Americas,Caribbean,Latin America & Caribbean,7,7,4,-99,1,BHS.geojson
4,"POLYGON ((-62.83193 17.87646, -62.84692 17.875...",3,Admin-0 country,6,France,FR1,1,2,Dependency,Saint Barthelemy,...,North America,Americas,Caribbean,Latin America & Caribbean,13,16,6,4,-99,BLM.geojson
5,"POLYGON ((-59.49331 13.08198, -59.52188 13.062...",1,Admin-0 country,5,Barbados,BRB,0,2,Sovereign country,Barbados,...,North America,Americas,Caribbean,Latin America & Caribbean,8,8,5,3,1,BRB.geojson
6,"POLYGON ((-83.64199 10.91724, -83.61729 10.877...",1,Admin-0 country,5,Costa Rica,CRI,0,2,Sovereign country,Costa Rica,...,North America,Americas,Central America,Latin America & Caribbean,10,10,4,-99,1,CRI.geojson
7,"MULTIPOLYGON (((-87.85293 17.42285, -87.92998 ...",1,Admin-0 country,6,Belize,BLZ,0,2,Sovereign country,Belize,...,North America,Americas,Central America,Latin America & Caribbean,6,6,6,-99,1,BLZ.geojson
8,"POLYGON ((-64.73027 32.29346, -64.82012 32.259...",3,Admin-0 country,6,United Kingdom,GB1,1,2,Dependency,Bermuda,...,North America,Americas,Northern America,North America,7,7,5,4,-99,BMU.geojson
9,"MULTIPOLYGON (((-82.56177 21.57168, -82.65483 ...",1,Admin-0 country,3,Cuba,CUB,0,2,Sovereign country,Cuba,...,North America,Americas,Caribbean,Latin America & Caribbean,4,4,4,-99,1,CUB.geojson


Draw the visualisation

In [41]:
alt.Chart(
    data=d_boxplot,
    height=750,
    width=1000,
    padding=20,
    title=alt.TitleParams(
        text='Age of Power Plants by Region',
        fontSize=22,
        font=__CHART_FONT__
    )
).mark_boxplot(
    size=50,
    extent=0.5,
    outliers=False,
    median=alt.MarkConfig(
        stroke='black'
    )
).encode(
    x=alt.X(
        'region:N',
        axis=alt.Axis(
            title="Region",
            titleFontSize=18,
            titleFont=__CHART_FONT__,
            labelFontSize=14,
            labelFont=__CHART_FONT__,
            labelAngle=45
        ),
        # # this sort stopped working for some reason smh my head
        # sort=alt.SortField(
        #     field='Renewable'
        # )
    ),
    y=alt.Y(
        'age:Q',
        axis=alt.Axis(
            title='Age (years)',
            titleFontSize=18,
            titleFont=__CHART_FONT__,
            labelFontSize=14,
            labelFont=__CHART_FONT__,
            tickCount=20
        )
    ),
    color=alt.Color(
        # colour boxes red/blue based on renewability
        'region:N',
        # scale=alt.Scale(
        #     range=['#FF5722', '#00BCD4']
        # ),
        legend=alt.Legend(
            titleFontSize=18,
            titleFont=__CHART_FONT__,
            labelFontSize=16,
            labelFont=__CHART_FONT__
        )
    )
)

alt.Chart(...)